# Задача обучение девочи

Нейронная сеть, решающая задачу оценки парня по трём критериям: наличие квартиры; привлекательность; любовь тяжёлого рока (негативный критерий).

Нейронная сеть должна выдавать результат в виде численных значений в интервале [0, 1], где 0 - отризательное отношение к парню; 1 - позитивное отношение.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Union

## Функция генерации топологии сети

In [2]:
def Sigmoid(u: Union[float, list[float]]): return 1 / (1 + np.exp(-u))
def dSigmoid(y: Union[float, list[float]]): return y * (1 - y)

In [3]:
def Topology(Inputs: int, layers: list[int]):
    '''
    Функция генерации топологии нейронной сети

    Parameters
    ----------
    Inputs: int: 
        число входов сети
    layers: list[int]: 
        список размеров слоёв (число нейронов в каждом)

    Returns
    -------
        Функция возвращает список структуры каждого слоя, содержащий кортежи: (w, o). 
        Где w - матрица передачи слоя, o - вектор смещения
    '''
    layersCount = len(layers)
    
    M = [Inputs] + layers

    np.random.seed(42)

    T = []
    for i in range(0, layersCount):
            w = np.random.normal(0, 1, size=(M[i+1], M[i]))
            o = np.random.normal(0, 1, size=(M[i+1]))
            T.append((w, o))

    return T

T = Topology(3, [2, 1])
T

[(array([[ 0.49671415, -0.1382643 ,  0.64768854],
         [ 1.52302986, -0.23415337, -0.23413696]]),
  array([1.57921282, 0.76743473])),
 (array([[-0.46947439,  0.54256004]]), array([-0.46341769]))]

## Алгоритм прямого распространения

In [4]:
def Forward(T, X: list[float]):
    x = np.array(X)
    for w, o in T:
        u = w @ x + o
        y = Sigmoid(u)
        x = y
    return y

Y2 = Forward(T, [1,1,-1])
Y2

array([0.41608565])

## Алгоритм обратного распространения

In [28]:
W1 = np.array([
    [-0.89254, -0.16455, -0.5034],
    [-1.78889, -1.87561, -1.20992]
])

O1 = np.array([
    1.34141, 
    0.06761
])

W2 = np.array([
    [0.93853, -0.10164]
])

O2 = np.array([
    0.56282
])

T0 = [
    (W1, O1),
    (W2, O2)
]

[(array([[-0.89254, -0.16455, -0.5034 ],
         [-1.78889, -1.87561, -1.20992]]),
  array([1.34141, 0.06761])),
 (array([[ 0.93853, -0.10164]]), array([0.56282]))]

In [22]:
def Test(x):
    return x * (1 - x)

def Backward(T, X: list[float], Y0: list[float], rho: float):
    layersCount = len(T)
    x = np.array(X)
    XX = [x]

    Y = []
    for w, o in T:
        u = w @ x + o
        y = Sigmoid(u)
        Y.append(y)
        x = y

    e = Y0 - Y[-1]
    XX += Y
    dsgm = np.vectorize(Test)
    T2 = T.copy()
    i = 0
    for i in range(layersCount-1, -1, -1):
        w, o = T[i]
        d = e * dsgm(Y[i])
        print(f"{i}:  e={e}")
        print(f" :  d={d}")
        print(f" : XX={XX[i]}")

        dw = d * XX[i].T
        print(f" : dw={dw}")
        print()
        e = w.T * d
        i += 1

    return 0

T2 = Backward(T, [1, 1, -1], [1], 0.5)
T2

1:  e=[0.58391435]
 :  d=[0.14186689]
 : XX=[0.78414281 0.90808286]
 : dw=[0.1112439  0.12882689]

0:  e=[[-0.06660287]
 [ 0.0769713 ]]
 :  d=[[-0.01127339 -0.00555923]
 [ 0.01302838  0.00642467]]
 : XX=[ 1  1 -1]


ValueError: operands could not be broadcast together with shapes (2,2) (3,) 